<a href="https://colab.research.google.com/github/jiheddachraoui/occupancy_grid_generator/blob/main/OG_fusion_saver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREPARATION

In [ ]:
import os
import sys
import csv
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from sklearn.linear_model import RANSACRegressor
from scipy import stats
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
datadir = '/content/drive/MyDrive/s_data/'
scenario_file='scenario4_gt/'
scenario='ground truth/'+scenario_file
output='output/'+scenario_file


for item in os.listdir(os.path.join(datadir,scenario)):
    # Construct the full path to the item
    item_path = os.path.join(os.path.join(datadir,scenario), item)
    # Check if the item is a directory
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        # Print the path of the subdirectory
        print(item_path)



/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_2 vehicle.mini.cooper_s
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_1 vehicle.tesla.model3


In [ ]:
ALPHA = 1
BHETA = 1*np.pi/180
RESOLUTION = 0.1
MAX_RANGE = 70
MAP_WIDTH = 100
SPHERICAL2CARTESIAN_BIAS = 6
### Some derived parameters
# OOR_MASK = A mask to filter out measurements that are out of MAX_RANGE
MAP_SIZE_X = int(MAP_WIDTH/RESOLUTION)
MAP_SIZE_Y = int(MAP_WIDTH/RESOLUTION)
xarr = np.arange(-MAP_WIDTH/2,MAP_WIDTH/2,RESOLUTION)
yarr = np.arange(-MAP_WIDTH/2,MAP_WIDTH/2,RESOLUTION)
MAP_XX, MAP_YY = np.meshgrid(xarr, -yarr)
rgrid = np.sqrt(np.add(np.square(MAP_XX),np.square(MAP_YY)))
OOR_MASK = rgrid >= MAX_RANGE
DYNAMIC_THRESHOLD=0.5

FREE_CONF = 0.7
OCC_CONF = 0.7
DYNAMIC_THRESHOLD = 0.4
dgm_time_0 = np.zeros((MAP_SIZE_Y,MAP_SIZE_X,3))
dgm_time_0[:,:,0] = 1

In [ ]:
def load_matrix(folder_path,timestamp):
  ts = str(timestamp)
  for file_name in os.listdir(folder_path):
          if os.path.splitext(file_name)[0] == ts:

              file_path = os.path.join(folder_path,file_name)
              #print(file_path)
              break

  m=np.load(file_path)
  return m

In [ ]:
def filter_road_points(lidar_in, is_road, threshold=0.5):
        y_true = lidar_in[:, 1]
        y = np.zeros_like(y_true)
        y = y_true * is_road  # set road points' predicted height to true height
        delta_y = np.absolute(y_true-y).flatten()
        is_not_road = delta_y > threshold
        lidar_out = lidar_in[is_not_road, :].copy()
        return lidar_out


In [ ]:
def generate_measurement_dgm(lidar_in,dgm_shape):
  ### Calculate the position of LiDAR points in spherical coordinate
  rphi_meas = np.zeros((lidar_in.shape[0],2))
  rphi_meas[:,1] = np.sqrt(np.add(np.square(lidar_in[:,0]),np.square(lidar_in[:,1])))/ALPHA
  rphi_meas[:,0] = (np.arctan2(lidar_in[:,1],lidar_in[:,0])+np.pi)/BHETA
  rphi_meas = np.unique(rphi_meas.astype(int),axis=0)
  rphi_meas = rphi_meas[rphi_meas[:,1]<int(MAX_RANGE/ALPHA),:]
  rphi_meas = rphi_meas[rphi_meas[:,0]<int(2*np.pi/BHETA),:]

  ### Initiate and fill the spherical scan grid
  sg_ang_bin = int(2*np.pi/BHETA)
  sg_rng_bin = int(MAX_RANGE/ALPHA)
  scan_grid = np.zeros((sg_ang_bin,sg_rng_bin,3))
  scan_grid[:,:,0] = 1
  scan_grid[tuple(rphi_meas.T)] = (1-OCC_CONF,OCC_CONF,0)
  for ang in range(sg_ang_bin):
    ang_arr = rphi_meas[rphi_meas[:,0]==ang,1]
    if len(ang_arr) == 0:
      scan_grid[ang,:] = (1-FREE_CONF,0,FREE_CONF)
    else:
      min_r = np.min(ang_arr)
      scan_grid[ang,:min_r] = (1-FREE_CONF,0,FREE_CONF)

  ### Convert the spherical scan grid to the cartesian one
  dgm_sz = (dgm_shape[1],dgm_shape[0])
  dgm_cen = (int(dgm_shape[1]/2),int(dgm_shape[0]/2))
  radius = (MAX_RANGE/RESOLUTION) + SPHERICAL2CARTESIAN_BIAS
  dgm_step = cv2.warpPolar(scan_grid,dgm_sz,dgm_cen,radius,cv2.WARP_INVERSE_MAP)
  dgm_step[OOR_MASK] = (1,0,0)
  dgm_step = cv2.rotate(dgm_step, cv2.ROTATE_90_CLOCKWISE)
  return dgm_step


In [ ]:
def update_dgm(prior_dgm,new_dgm):
  ### Calculate conflicting mass
  conflict_mass = np.multiply(prior_dgm[:,:,2],new_dgm[:,:,1])
  conflict_mass = np.add(conflict_mass,np.multiply(prior_dgm[:,:,1],new_dgm[:,:,2]))

  ### Calculate free mass
  free_mass = np.multiply(prior_dgm[:,:,0],new_dgm[:,:,2])
  free_mass = np.add(free_mass,np.multiply(prior_dgm[:,:,2],new_dgm[:,:,0]))
  free_mass = np.add(free_mass,np.multiply(prior_dgm[:,:,2],new_dgm[:,:,2]))
  free_mass = np.divide(free_mass,1-conflict_mass)

  ### Calculate occupied mass
  occ_mass = np.multiply(prior_dgm[:,:,0],new_dgm[:,:,1])
  occ_mass = np.add(occ_mass,np.multiply(prior_dgm[:,:,1],new_dgm[:,:,0]))
  occ_mass = np.add(occ_mass,np.multiply(prior_dgm[:,:,1],new_dgm[:,:,1]))
  occ_mass = np.divide(occ_mass,1-conflict_mass)

  ### Calculate unknown mass
  unknown_mass = np.multiply(prior_dgm[:,:,0],new_dgm[:,:,0])
  unknown_mass = np.divide(unknown_mass,1-conflict_mass)

  updated_dgm = np.stack((unknown_mass,occ_mass,free_mass),axis=2)
  return updated_dgm,conflict_mass

### Convert the DGM to a displayable figure
def predict_dgm(dgm,dynamic_mass,dt=0.5):
  max_mass = np.argmax(dgm,axis=2)
  pred_map = np.zeros(dgm.shape)
  # The unknown cells: gray
  pred_map[max_mass==0] = (123,123,123)
  # The occupied cells: black
  pred_map[max_mass==1] = (0,0,0)
  # The free cells: white
  pred_map[max_mass==2] = (255,255,255)
  # The dynamic cells: blue
  pred_map[dynamic_mass>=dt] = (0,0,0)
  return pred_map.astype(np.uint8)


In [ ]:
def shift_pose_dgm(dgm, init, fin):
  dgm_o = dgm.copy()
  #theta = init[2] /180 * np.pi
  theta = -init[2]
  rot_m = np.array([[np.cos(theta),np.sin(theta)],[-np.sin(theta),np.cos(theta)]])
  trs_m = np.array([[init[0]],[init[1]]])
  point = np.array(fin[:2]).reshape((-1,1))
  point_1 = (point - trs_m)
  point_2 = np.dot(rot_m,-point_1)
  delta_theta = (fin[2] - init[2])
  delta = np.array([point_2[1,0]/RESOLUTION,point_2[0,0]/RESOLUTION,0])

  M = np.array([[1,0,delta[0]],[0,1,-delta[1]]])
  dst = cv2.warpAffine(dgm_o,M,(dgm_o.shape[1],dgm_o.shape[0]),borderValue=0.5)
  M = cv2.getRotationMatrix2D((dgm_o.shape[1]/2+0.5,dgm_o.shape[0]/2+0.5),-delta_theta* 180 / np.pi,1)
  #M = cv2.getRotationMatrix2D((dgm_o.shape[1]/2+0.5,dgm_o.shape[0]/2+0.5),delta_theta,1)
  dst = cv2.warpAffine(dst,M,(dgm_o.shape[1],dgm_o.shape[0]),borderValue=0.5)
  return dst

In [ ]:
vehicles=[]
for item in os.listdir(os.path.join(datadir,scenario)):
    # Construct the full path to the item
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        folder_path = os.path.join(os.path.join(datadir,scenario), item)
        output_path = os.path.join(datadir,output)
        lidar_path=folder_path+'/Lidar'
        img_path=folder_path+'/bird_eye'
        save_path=output_path+'OG fusion'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        num_files = len([f for f in os.listdir(lidar_path) if os.path.isfile(os.path.join(lidar_path, f))])
        print('Number of files in {} :'.format(lidar_path), num_files)
        vehicles.append((folder_path,num_files,save_path))

Number of files in /content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/Lidar : 89
Number of files in /content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_2 vehicle.mini.cooper_s/Lidar : 89
Number of files in /content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_1 vehicle.tesla.model3/Lidar : 89


In [ ]:
vehicles

[('/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2',
  89,
  '/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion'),
 ('/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_2 vehicle.mini.cooper_s',
  89,
  '/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion'),
 ('/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_1 vehicle.tesla.model3',
  89,
  '/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion')]

In [ ]:
folder_paths=[]

for i, vehicle in enumerate(vehicles, start=1):
    variable_name = f'folder_path{i}'
    folder_path = os.path.join(datadir, scenario, vehicle[0])
    globals()[variable_name] = folder_path
    folder_paths.append(folder_path)
nbr_frames=[]
for i, vehicle in enumerate(vehicles, start=1):

    nbr = vehicle[1]

    nbr_frames.append(nbr)
nbr=min(nbr_frames)

In [ ]:
folder_paths

['/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2',
 '/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_2 vehicle.mini.cooper_s',
 '/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_1 vehicle.tesla.model3']

In [ ]:
'''
import random
seed_value = 13

# Set the seed for the random number generator
random.seed(seed_value)
# Randomly permute the order of folder_paths
random.shuffle(folder_paths)
print(folder_paths)'''

'\nimport random\nseed_value = 13\n\n# Set the seed for the random number generator\nrandom.seed(seed_value)\n# Randomly permute the order of folder_paths\nrandom.shuffle(folder_paths)\nprint(folder_paths)'

In [ ]:
folder_paths

['/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2',
 '/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_2 vehicle.mini.cooper_s',
 '/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_1 vehicle.tesla.model3']

In [ ]:
print(nbr)

89


In [ ]:
for date in range(nbr):

  for i, folder_path in enumerate(folder_paths, start=1):

      save_path=vehicles[i-1][2]+'/fusion_image'
      save_path1=vehicles[i-1][2]+'/mass_function'
      save_path2=vehicles[i-1][2]+'/conflict'
      if not os.path.exists(save_path):
        os.makedirs(save_path)
      if not os.path.exists(save_path1):
        os.makedirs(save_path1)
      if not os.path.exists(save_path2):
        os.makedirs(save_path2)
      transformation_matrices_path = os.path.join(folder_path, "pos_matrices/lidar_2_world")
      transformation_matrices_path_variable = f'transformation_matrices_path{i}'
      globals()[transformation_matrices_path_variable] = transformation_matrices_path



      transformation_matrix_variable = f'transform{i}'
      globals()[transformation_matrix_variable] = load_matrix(transformation_matrices_path, date)
      transform = globals()[transformation_matrix_variable]


      location_variable = f'loc{i}'
      globals()[location_variable] = transform[:3, 3:].flatten().tolist()

      rotation_path = os.path.join(folder_path, "pos_matrices/rotation_pyr")
      rotation_path_variable = f'rot_path{i}'
      globals()[rotation_path_variable] = rotation_path

      rotation_variable = f'rot{i}'
      globals()[rotation_variable] = load_matrix(rotation_path, date)



      R = np.array(transform[:3, :3])
      pitch = np.arcsin(R[2, 0])
      yaw = -np.arctan2(R[1, 0], R[0, 0])
      roll = -np.arctan2(R[2, 1], R[2, 2])
      pose_variable = f'pose{i}'
      globals()[pose_variable] = np.array([globals()[location_variable][0], globals()[location_variable][1], yaw])


      lidar_variable = f'lidar{i}'
      lidar_path = os.path.join(folder_path, "Lidar")
      lidar_file = os.path.join(lidar_path, f'{date}.npy')

      lidar_data = np.load(lidar_file)
      globals()[lidar_variable] = lidar_data.reshape(-1, 6)

      lidar_raw_variable = f'lidar_raw{i}'
      globals()[lidar_raw_variable] = globals()[lidar_variable].reshape(-1, 6)

      globals()[lidar_raw_variable] = globals()[lidar_raw_variable][globals()[lidar_raw_variable][:, 2] <= 0, :]

      radius = 2.7
      distances_variable = f'distances{i}'
      distances = np.sqrt((globals()[lidar_raw_variable][:, 0])**2 + (globals()[lidar_raw_variable][:, 1])**2)
      globals()[distances_variable] = distances

      globals()[lidar_raw_variable] = globals()[lidar_raw_variable][globals()[distances_variable] > radius, :]

      labels_variable = f'labels{i}'
      labels = np.array(globals()[lidar_raw_variable][:, 5]).astype(int)
      globals()[labels_variable] = labels

      lidar_labeled_variable = f'lidar_labeled{i}'
      lidar_labeled = np.array(globals()[lidar_raw_variable][..., [0, 1, 2, 5]])
      globals()[lidar_labeled_variable] = lidar_labeled

      globals()[lidar_raw_variable] = globals()[lidar_raw_variable][:, :3]

      road_filter_variable = f'road_filter{i}'
      road_filter = np.zeros_like(globals()[labels_variable])
      road_filter[(globals()[labels_variable] == 0) | (globals()[labels_variable] == 7) | (globals()[labels_variable] == 6)] = 1
      globals()[road_filter_variable] = road_filter

      lidar_nonroad_variable = f'lidar_nonroad{i}'
      globals()[lidar_nonroad_variable] = filter_road_points(globals()[lidar_raw_variable], globals()[road_filter_variable], 0.1)
      lidar_nonroad_variable = f'lidar_nonroad{i}'
      lidar_dgm_variable = f'lidar_dgm{i}'

      globals()[lidar_dgm_variable] = globals()[lidar_nonroad_variable][:, [0, 1]]
      lidar_dgm_variable = f'lidar_dgm{i}'
      dgm_variable = f'dgm{i}'

      globals()[dgm_variable] = generate_measurement_dgm(globals()[lidar_dgm_variable], dgm_time_0.shape)

      dgm_time_variable = f'dgm_time_{i}'
      dynamic_mass_variable = f'dynamic_mass_{i}'
      dgm_pred_variable = f'dgm_pred_{i}'

      globals()[dgm_time_variable], globals()[dynamic_mass_variable] = update_dgm(dgm_time_0, globals()[dgm_variable])
      globals()[dgm_pred_variable] = predict_dgm(globals()[dgm_time_variable], globals()[dynamic_mass_variable])

  shift_dgm_time_1 = shift_pose_dgm(dgm_time_1,pose1,pose2)
  dgm_fuse,dynamic_mass_fuse = update_dgm(shift_dgm_time_1,dgm_time_2)
  dgm_pred_fuse = predict_dgm(dgm_fuse,dynamic_mass_fuse,0.5)
  if 'pose3' in globals():
      shift_dgm_time_3 = shift_pose_dgm(dgm_time_3, pose3, pose2)
      dgm_fuse,dynamic_mass_fuse = update_dgm(dgm_fuse,shift_dgm_time_3)
      dgm_pred_fuse = predict_dgm(dgm_fuse,dynamic_mass_fuse,0.5)

  if 'pose4' in globals():
    shift_dgm_time_4 = shift_pose_dgm(dgm_time_4, pose4, pose2)
    dgm_fuse,dynamic_mass_fuse = update_dgm(dgm_fuse,shift_dgm_time_4 )
    dgm_pred_fuse = predict_dgm(dgm_fuse,dynamic_mass_fuse,0.5)
  else:
    continue
  rgb_image = cv2.cvtColor(dgm_pred_fuse, cv2.COLOR_BGR2RGB)
  cv2.imwrite(save_path+'/{}.png'.format(date),rgb_image)
  print(save_path+'/{}.png'.format(date))
  np.save(save_path1+'/{}.npy'.format(date),dgm_fuse)
  print(save_path1+'/{}.npy'.format(date))
  np.save(save_path2+'/{}.npy'.format(date),dynamic_mass_fuse)
  print(save_path2+'/{}.npy'.format(date))

/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/fusion_image/0.png
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/mass_function/0.npy
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/conflict/0.npy
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/fusion_image/1.png
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/mass_function/1.npy
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/conflict/1.npy
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/fusion_image/2.png
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/mass_function/2.npy
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/conflict/2.npy
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/fusion_image/3.png
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/mass_function/3.npy
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/conflict/3.npy
/content/drive/MyDrive/s_data/output/scenario4_gt/OG fusion/fusi